In [19]:
import json
import os
import os.path as osp
import sys
from PIL import Image
import pickle as pkl
import cv2
import pandas as pd

from detectron.datasets.json_dataset import JsonDataset
from detectron.utils.io import load_object, save_object
from detectron.datasets.roidb import combined_roidb_for_training
import detectron.utils.vis as vis_utils
from detectron.core.config import cfg

taxonomyDate = '20191017'
mappingDate = taxonomyDate #'201909' # where the {dataset}_mapping_to_GOD.pkl is

godLabelFile = '/media/data/chnxi/GOD/taxonomy/GOD_taxonomy_{}.tsv'.format(taxonomyDate)
godLabels = [l.rstrip().split('\t') for l in open(godLabelFile,'r').readlines()[1:]]
print (godLabels[-10:])

god_data = pd.read_csv(godLabelFile, delimiter='\t')
god_name_id_map = dict(zip(god_data['GOD_v1_name'], god_data['GOD_v1_id']))
god_classes = list(god_data['GOD_v1_name'])
print(god_classes[-10:])

def checkMkdir(dirname):
    if not osp.isdir(dirname):
        os.makedirs(dirname)
        
outValFolder = '/media/data/chnxi/GOD/json_annotations/val_{}'.format(taxonomyDate)

checkMkdir(outValFolder)
        
datasetList = ['Object365'] #['COCO_91', 'FashionV2', 'HomeFurniture',  'OpenImage', 'Object365'] # #, 


mappingFormat = '/media/data/chnxi/GOD/mappings/{}/dataset_nameid_to_GOD_nameid/{}_mapping_to_GOD.pkl'.format(mappingDate, '{}')

dataPath = '/media/data/chnxi/'
valSetFiles = {'COCO_91': dataPath + 'coco/annotations/instances_minival2014.json',
                'FashionV2': dataPath + 'FashionV2/json_annotations/val_annotations.json',
                'HomeFurniture': dataPath + 'HomeFurniture/json_annotations/val_annotations.json',
                'OpenImage': dataPath + 'OpenImage/json_annotations/validation_annotations.json',
                'Object365': dataPath + 'Object365/json_annotations/objects365_val_TrainCategs.json'
               }

[['837', 'text/a_region_of_text_in_image'], ['838', 'tools/lawnmower'], ['839', 'tools/leafblower'], ['840', 'toy/arts_and_craft'], ['841', 'toy/building_toy'], ['842', 'vehicle/land_vehicle/scooter'], ['843', 'vehicle/land_vehicle/tractor'], ['844', 'visual_signs/electronics_signs'], ['845', 'visual_signs/fabric_care_labels'], ['846', 'visual_signs/indoor_signs']]
['text/a_region_of_text_in_image', 'tools/lawnmower', 'tools/leafblower', 'toy/arts_and_craft', 'toy/building_toy', 'vehicle/land_vehicle/scooter', 'vehicle/land_vehicle/tractor', 'visual_signs/electronics_signs', 'visual_signs/fabric_care_labels', 'visual_signs/indoor_signs']


In [20]:
for cls, id in god_name_id_map.items():
    print('{}\t{}'.format(id, cls))

1	animal/amphibian/frog
2	animal/arthropod/arachnid/scorpion
3	animal/arthropod/arachnid/spider
4	animal/arthropod/crustacean/crab
5	animal/arthropod/crustacean/lobster
6	animal/arthropod/crustacean/shrimp
7	animal/arthropod/insect/ant
8	animal/arthropod/insect/bee
9	animal/arthropod/insect/beetle
10	animal/arthropod/insect/butterfly
11	animal/arthropod/insect/caterpillar
12	animal/arthropod/insect/dragonfly
13	animal/arthropod/insect/ladybug
14	animal/arthropod/insect/moths_and_butterflies
15	animal/arthropod/insect/tick
16	animal/arthropod/isopod
17	animal/arthropod/myriapod/centipede
18	animal/bird
19	animal/bird/blue_jay
20	animal/bird/canary
21	animal/bird/chicken
22	animal/bird/duck
23	animal/bird/eagle
24	animal/bird/falcon
25	animal/bird/goose
26	animal/bird/magpie
27	animal/bird/ostrich
28	animal/bird/owl
29	animal/bird/parrot
30	animal/bird/penguin
31	animal/bird/raven
32	animal/bird/sparrow
33	animal/bird/swan
34	animal/bird/turkey
35	animal/bird/woodpecker
36	animal/dinosau

In [16]:
dataset = 'Object365'
mapFile = mappingFormat.format(dataset)
mapping = pkl.load(open(mapFile,'rb'))
print(mapping.keys())
print(mapping['name_to_god_name'])
# print (mapping['name_to_god_name'])

dict_keys(['name_to_god_name', 'id_map', 'name_to_god_id'])
{'red cabbage': 'food_or_drink/vegetable_or_fruit/round_cabbage_like_leafy_vegetable/cabbage', 'skis': 'sports_equipment/ski', 'microwave': 'electronics/home_appliances/microwave', 'chair': 'home_or_office_furnishing_or_decor/seating/chair', 'dumbbell': 'sports_equipment/dumbbell', 'sports car': 'vehicle/land_vehicle/car/sports_car', 'carpet': 'home_or_office_furnishing_or_decor/rug_or_mat', 'trolley': 'vehicle/land_vehicle/trolley', 'tv': 'electronics/home_entertainment/tv', 'swan': 'animal/bird/swan', 'rice cooker': 'electronics/home_appliances/electronic_pressure_cooker', 'screwdriver': 'tools/screwdriver', 'vase': 'home_or_office_furnishing_or_decor/vase_or_pot', 'spoon': 'home_or_office_furnishing_or_decor/dinnerware_or_serveware/spoon', 'fan': 'electronics/home_appliances/fan', 'llama': 'animal/mammal/llama', 'stapler': 'school_or_office_supply/stapler', 'parrot': 'animal/bird/parrot', 'tricycle': 'vehicle/land_vehicle/t

In [23]:
updateAnno = True
for dataset in datasetList:
    print ("dataset = {}".format(dataset))
    outFileName = osp.join(outValFolder, 'GOD_{}_val_annotations.json'.format(dataset))
    print ("outFileName = {}".format(outFileName))
    mapFile = mappingFormat.format(dataset)
    mapping = pkl.load(open(mapFile,'rb'))
    #print (mapping.keys())
    id_map = mapping['id_map']
    name_map = mapping['name_to_god_name']
    
    annoFile = valSetFiles[dataset]
    annoData = json.load(open(annoFile,'r'))
    #print (anno.keys())
    
    
    godCateg = []
    supercat = 'GOD_{}'.format(dataset)
    for god_categ in godLabels:
        cls = god_categ[1] #[0]
        id = int(god_categ[0])
        cats = {'supercategory':supercat,
                'id':id,
                'name':cls}
        godCateg.append(cats) 
        
        
    subsetCateg = []
    for org_cat in annoData['categories']:
        org_id = org_cat['id']
        if org_id in id_map:
            god_id = id_map[org_id]
            org_name = org_cat['name']
            org_name = org_name.strip().lower()
            if dataset == 'OpenImage':
                org_name = org_name.lower().replace('_',' ')
            cats = {'supercategory':supercat,
                    'id':god_id,
                    'name':name_map[org_name]}
            subsetCateg.append(cats)
    
    
    
    #################################################################################
#     for ix, categ in enumerate(annoData['categories']):
#         src_id = categ['id']
#         if src_id in id_map:
#             categ['id'] = id_map[src_id]
# #             print ("{}==>{}".format(src_id, id_map[src_id]))
# #             print (categ)
#             newCateg.append(categ)
#         else:
#             print ("{} not in GOD".format(categ['name']))
    #################################################################################
    
    print(godCateg[:10])
    print (subsetCateg[:10])
    annoData['categories'] = godCateg
    annoData['subset_categories'] = subsetCateg
    
    #################################################################################
    if osp.exists(outFileName) and not updateAnno:
        godAnnoData = json.load(open(outFileName,'r'))
        annoData['annotations'] = godAnnoData['annotations']
    else: # if not exists(outFileName) or  updateAnno
        newAnnos = []
        for ox, obj in enumerate(annoData['annotations']):
            if obj['category_id'] in id_map:
                src_id = obj['category_id']
                god_id = id_map[obj['category_id']]
                obj['category_id'] = god_id
                newAnnos.append(obj)
                if (ox + 1) % 10000 == 0:
                    print ("{}==>{}".format(src_id, god_id))
    #                 print (obj)
        annoData['annotations'] = newAnnos
    print ("saving to {}".format(outFileName))
    with open(outFileName,'w') as fout:
        json.dump(annoData, fout)
    print ("Done")
    

dataset = Object365
outFileName = /media/data/chnxi/GOD/json_annotations/val_20191017/GOD_Object365_val_annotations.json
[{'supercategory': 'GOD_Object365', 'id': 1, 'name': 'animal/amphibian/frog'}, {'supercategory': 'GOD_Object365', 'id': 2, 'name': 'animal/arthropod/arachnid/scorpion'}, {'supercategory': 'GOD_Object365', 'id': 3, 'name': 'animal/arthropod/arachnid/spider'}, {'supercategory': 'GOD_Object365', 'id': 4, 'name': 'animal/arthropod/crustacean/crab'}, {'supercategory': 'GOD_Object365', 'id': 5, 'name': 'animal/arthropod/crustacean/lobster'}, {'supercategory': 'GOD_Object365', 'id': 6, 'name': 'animal/arthropod/crustacean/shrimp'}, {'supercategory': 'GOD_Object365', 'id': 7, 'name': 'animal/arthropod/insect/ant'}, {'supercategory': 'GOD_Object365', 'id': 8, 'name': 'animal/arthropod/insect/bee'}, {'supercategory': 'GOD_Object365', 'id': 9, 'name': 'animal/arthropod/insect/beetle'}, {'supercategory': 'GOD_Object365', 'id': 10, 'name': 'animal/arthropod/insect/butterfly'}]
[{